# Package

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [2]:
path = 'input/store-sales-time-series-forecasting/'

# Feature Engineering

In [3]:
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

## Oil

In [4]:
df_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
df_oil['ma_oil'] = df_oil['dcoilwtico'].rolling(7).mean()
calendar = calendar.merge(df_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='bfill', inplace=True)
calendar.head()

,dcoilwtico,ma_oil
2013-01-01,NaN,93.218571
2013-01-02,93.14,93.218571
2013-01-03,92.97,93.218571
2013-01-04,93.12,93.218571
2013-01-05,NaN,93.218571


## Day of week

In [5]:
calendar['dow'] = calendar.index.dayofweek
calendar['dom'] = calendar.index.day
calendar.head()

,dcoilwtico,ma_oil,dow,dom
2013-01-01,NaN,93.218571,1,1
2013-01-02,93.14,93.218571,2,2
2013-01-03,92.97,93.218571,3,3
2013-01-04,93.12,93.218571,4,4
2013-01-05,NaN,93.218571,5,5


## Hoilday

In [6]:
holidays = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
holidays = holidays.set_index('date').sort_index()
holidays = holidays[holidays.locale == 'National'] # National level only for simplicity
#holidays = holidays.groupby(holidays.index).first() # Keep one event only
holidays.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


## Wage Day

In [7]:
df = pd.DataFrame(index = pd.date_range('2013-01-01', '2017-08-31')).to_period('D')
df['wageday']=0
df.loc[(df.index.to_timestamp().is_month_end) | (df.index.day == 15), 'wageday'] = 1
calendar['wageday'] = df['wageday'].values
calendar

,dcoilwtico,ma_oil,dow,dom,wageday
2013-01-01,NaN,93.218571,1,1,0
2013-01-02,93.14,93.218571,2,2,0
2013-01-03,92.97,93.218571,3,3,0
2013-01-04,93.12,93.218571,4,4,0
2013-01-05,NaN,93.218571,5,5,0
...,...,...,...,...,...
2017-08-27,NaN,47.624286,6,27,0
2017-08-28,46.40,47.624286,0,28,0
2017-08-29,46.46,47.320000,1,29,0
2017-08-30,45.96,47.115714,2,30,0


## Work Day

In [8]:
calendar['wd'] = True
calendar.loc[calendar.dow > 4, 'wd'] = False
calendar = calendar.merge(holidays, how='left', left_index=True, right_index=True)
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True
calendar.head()

,dcoilwtico,ma_oil,dow,dom,wageday,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,93.218571,1,1,0,False,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,93.14,93.218571,2,2,0,True,NaN,NaN,NaN,NaN,NaN
2013-01-03,92.97,93.218571,3,3,0,True,NaN,NaN,NaN,NaN,NaN
2013-01-04,93.12,93.218571,4,4,0,True,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,93.218571,5,5,0,True,Work Day,National,Ecuador,Recupero puente Navidad,False


## Label encoding

In [9]:
label_encoder = preprocessing.LabelEncoder()
calendar['type']= label_encoder.fit_transform(calendar['type'])
calendar

,dcoilwtico,ma_oil,dow,dom,wageday,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,93.218571,1,1,0,False,3,National,Ecuador,Primer dia del ano,False
2013-01-02,93.14,93.218571,2,2,0,True,6,NaN,NaN,NaN,NaN
2013-01-03,92.97,93.218571,3,3,0,True,6,NaN,NaN,NaN,NaN
2013-01-04,93.12,93.218571,4,4,0,True,6,NaN,NaN,NaN,NaN
2013-01-05,NaN,93.218571,5,5,0,True,5,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...,...,...,...,...
2017-08-27,NaN,47.624286,6,27,0,False,6,NaN,NaN,NaN,NaN
2017-08-28,46.40,47.624286,0,28,0,True,6,NaN,NaN,NaN,NaN
2017-08-29,46.46,47.320000,1,29,0,True,6,NaN,NaN,NaN,NaN
2017-08-30,45.96,47.115714,2,30,0,True,6,NaN,NaN,NaN,NaN


## Drop 12/25 which training set exclude

In [10]:
calendar = calendar.reset_index()
calendar.drop_duplicates(subset=['index'], keep='last', inplace=True, ignore_index=True)
calendar = calendar.set_index('index')
calendar = calendar.drop(index=['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'])
print(calendar.shape)

(1700, 11)


# Training

In [11]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()
df_train.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

## Label (y)

In [12]:
sdate = '2013-01-01'
edate = '2017-08-15'
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]
print(y.shape)
y.head()

(1684, 1782)


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2013-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2013-01-02        2.0       0.0    2.0    1091.0   0.0   470.652008   
2013-01-03        3.0       0.0    0.0     919.0   0.0   310.654999   
2013-01-04        3.0       0.0    3.0     953.0   0.0   198.365997   
2013-01-05        5.0       0.0    3.0    1160.0   0.0   301.057007   

                                                    ...                        \
store_nbr                                           ...         9               
family     CELEBRATION CLEANING  DAIRY        DELI  ... MAGAZINES       MEATS   
date                                                ...                         
2013-01-01         0.0      0.0    0.0    0.000000  ...       0.0    0.000000   
2013-01-02         0.0   1060.0  579.0  164.069000  ...       0.0  374.531006   
2013-01-03         0.0    836.0  453.0  151.582001  ...       0.0  400.863007   
2013-01-04         0.0    827.0  460.0  131.410995  ...       0.0  310.877991   
2013-01-05         0.0    811.0  464.0  118.612999  ...       0.0  447.684998   

                                                                           \
store_nbr                                                                   
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS     POULTRY   
date                                                                        
2013-01-01           0.0          0.0                     0.0    0.000000   
2013-01-02         482.0          0.0                     0.0  651.291992   
2013-01-03         372.0          0.0                     0.0  509.496002   
2013-01-04         324.0          0.0                     0.0  332.671997   
2013-01-05         461.0          0.0                     0.0  510.919006   

                                                                         
store_nbr                                                                
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                                                     
2013-01-01            0.0     0.0                        0.0   0.000000  
2013-01-02           83.0     0.0                        0.0  29.214001  
2013-01-03           66.0     0.0                        0.0  25.000000  
2013-01-04           57.0     0.0                        0.0  11.000000  
2013-01-05           84.0     0.0                        0.0  28.361000  

[5 rows x 1782 columns]

## Data point (X)

In [13]:
dp = DeterministicProcess(index = y.index, constant=False, order=1)
X = dp.in_sample()

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['dow'] = calendar.loc[sdate:edate]['dow'].values
X['dom'] = calendar.loc[sdate:edate]['dom'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['type'] = calendar.loc[sdate:edate]['type'].values
X['wageday'] = calendar.loc[sdate:edate]['wageday'].values
print(X.shape)
X.head()


(1684, 7)


,trend,oil,dow,dom,wd,type,wageday
date,,,,,,,
2013-01-01,1.0,93.218571,1,1,False,3,0
2013-01-02,2.0,93.218571,2,2,True,6,0
2013-01-03,3.0,93.218571,3,3,True,6,0
2013-01-04,4.0,93.218571,4,4,True,6,0
2013-01-05,5.0,93.218571,5,5,True,5,0


## Train

In [14]:
# model = LinearRegression()
# model.fit(X,y)
model_rf = RandomForestRegressor(n_estimators=200, random_state=2022, verbose=0)
model_rf.fit(X, y)

RandomForestRegressor(n_estimators=200, random_state=2022)

In [15]:
y_pred = pd.DataFrame(model_rf.predict(X), index=X.index, columns=y.columns)
y_pred=y_pred.stack(['store_nbr', 'family']).reset_index()
y_pred.head()

,date,store_nbr,family,sales
0,2013-01-01,1,AUTOMOTIVE,0.520
1,2013-01-01,1,BABY CARE,0.000
2,2013-01-01,1,BEAUTY,0.580
3,2013-01-01,1,BEVERAGES,279.975
4,2013-01-01,1,BOOKS,0.000


In [16]:
y_target=y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0
print(np.sqrt(mean_squared_log_error(y_target['sales'], y_target['sales_pred'])))

0.3830488471113362


In [17]:
y_target.groupby('family').apply(lambda x: np.sqrt(mean_squared_log_error(x['sales'], x['sales_pred'])))

family
AUTOMOTIVE                    0.265037
BABY CARE                     0.086514
BEAUTY                        0.229312
BEVERAGES                     0.567267
BOOKS                         0.059915
BREAD/BAKERY                  0.420421
CELEBRATION                   0.263912
CLEANING                      0.502605
DAIRY                         0.444676
DELI                          0.362789
EGGS                          0.331696
FROZEN FOODS                  0.339410
GROCERY I                     0.632159
GROCERY II                    0.306142
HARDWARE                      0.234399
HOME AND KITCHEN I            0.343756
HOME AND KITCHEN II           0.268375
HOME APPLIANCES               0.157176
HOME CARE                     0.642319
LADIESWEAR                    0.253634
LAWN AND GARDEN               0.262619
LINGERIE                      0.321476
LIQUOR,WINE,BEER              0.711298
MAGAZINES                     0.213218
MEATS                         0.361548
PERSONAL CARE     

# Prediction

In [18]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()
df_test['sales'] = 0
df_test.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2017-08-16      0
                     2017-08-17      0
                     2017-08-18      0
                     2017-08-19      0
                     2017-08-20      0

## Label (y_test)

In [19]:
stest = '2017-08-16'
etest = '2017-08-31'
y_test = df_test.unstack(['store_nbr', 'family']).loc[stest:etest]
print(y_test.shape)
y_test.head()

(16, 1782)


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-08-16          0         0      0         0     0            0   
2017-08-17          0         0      0         0     0            0   
2017-08-18          0         0      0         0     0            0   
2017-08-19          0         0      0         0     0            0   
2017-08-20          0         0      0         0     0            0   

                                            ...                                \
store_nbr                                   ...         9                       
family     CELEBRATION CLEANING DAIRY DELI  ... MAGAZINES MEATS PERSONAL CARE   
date                                        ...                                 
2017-08-16           0        0     0    0  ...         0     0             0   
2017-08-17           0        0     0    0  ...         0     0             0   
2017-08-18           0        0     0    0  ...         0     0             0   
2017-08-19           0        0     0    0  ...         0     0             0   
2017-08-20           0        0     0    0  ...         0     0             0   

                                                                        \
store_nbr                                                                
family     PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY PREPARED FOODS   
date                                                                     
2017-08-16            0                       0       0              0   
2017-08-17            0                       0       0              0   
2017-08-18            0                       0       0              0   
2017-08-19            0                       0       0              0   
2017-08-20            0                       0       0              0   

                                                       
store_nbr                                              
family     PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                   
2017-08-16       0                          0       0  
2017-08-17       0                          0       0  
2017-08-18       0                          0       0  
2017-08-19       0                          0       0  
2017-08-20       0                          0       0  

[5 rows x 1782 columns]

## Data point (X_test)

In [20]:
X_test = dp.out_of_sample(steps=16)

# Extentions
X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['dow'] = calendar.loc[stest:etest]['dow'].values
X_test['dom'] = calendar.loc[stest:etest]['dom'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values
X_test['type'] = calendar.loc[stest:etest]['type'].values
X_test['wageday'] = calendar.loc[stest:etest]['wageday'].values
X_test.index.names = ['date']
print(X_test.shape)
X_test

(16, 7)


,trend,oil,dow,dom,wd,type,wageday
date,,,,,,,
2017-08-16,1685.0,48.281429,2,16,True,6,0
2017-08-17,1686.0,47.995714,3,17,True,6,0
2017-08-18,1687.0,47.852857,4,18,True,6,0
2017-08-19,1688.0,47.688571,5,19,False,6,0
2017-08-20,1689.0,47.688571,6,20,False,6,0
2017-08-21,1690.0,47.688571,0,21,True,6,0
2017-08-22,1691.0,47.522857,1,22,True,6,0
2017-08-23,1692.0,47.645714,2,23,True,6,0
2017-08-24,1693.0,47.598571,3,24,True,6,0


## Predict

In [21]:
sales_pred = pd.DataFrame(model_rf.predict(X_test), index=X_test.index, columns=y_test.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family']).reset_index()
sales_pred['sales'] = sales_pred['sales'].clip(0.) # Sales should be >= 0
sales_pred.head()

,date,store_nbr,family,sales
0,2017-08-16,1,AUTOMOTIVE,5.495
1,2017-08-16,1,BABY CARE,0.000
2,2017-08-16,1,BEAUTY,3.745
3,2017-08-16,1,BEVERAGES,2365.295
4,2017-08-16,1,BOOKS,0.300


In [22]:
df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred['sales'].values
df_sub.to_csv('submission.csv', index=True)